# Constraints

### calculating by my functions

The constraints transform an ill-posed problem into a well-posed problem. So, we formulate a constrained non-linear inversion by summing constraints to the misfit function $\Psi (\textbf{m})$. 

\begin{equation}
\Gamma (\textbf{m}) = \Psi (\textbf{m}) + \mu \sum\limits^{6}_{j=1}\alpha_{j}\varphi_{j}(\textbf{m})
\end{equation}

where $\textbf{m}$ is the parameter vector, $\varphi_{j}$ is the $j$th constraint, $\alpha_{j}$ is the $j$-th is a positive scalar that defines the weight of the $j$-th constraint and $\mu$ is a positive scalar called regularization parameter. To solve this problem we will use the Levenberg-Marquadt method.

In this code, we impose six constraints to an Hessian matrix and parameter vector. The functions are tested in some special cases.

### Import required library

In [1]:
import numpy as np
import polyprism_tests as tests

/home/vital/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


### Import my functions from an external file

In [2]:
import mag_polyprism_functions as mfun

In [26]:
# building the diagonals
d0 = np.zeros(M+2)
d0[:M] = 2.*alpha
d0 = np.resize(d0, P)
#d0 = np.hstack((d0,np.zeros(1)))

d1 = np.zeros(M+2)
d1[:M-1] = - alpha
d1 = np.resize(d1, P-1)

dM = np.zeros(M+2)
dM[0] = - alpha
dM = np.resize(dM, P-M+1)

print d0
print d1
print dM

[ 2.  2.  2.  2.  0.  0.  2.  2.  2.  2.  0.  0.  2.  2.  2.  2.  0.  0.]
[-1. -1. -1.  0.  0.  0. -1. -1. -1.  0.  0.  0. -1. -1. -1.  0.  0.]
[-1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0. -1.  0.  0.]


In [27]:
d0 = np.insert(d0, -1, 0.)
print d0

[ 2.  2.  2.  2.  0.  0.  2.  2.  2.  2.  0.  0.  2.  2.  2.  2.  0.  0.
  0.]


In [ ]:
np.zeros((3,3))

### Input

In [24]:
M = 4 # number of vertices
L = 3 # number of prisms
P = L*(M+2) # number of parameters

In [25]:
alpha = 1.0 # weight

In [5]:
H1 = np.zeros((P,P)) # Hessian matrix
H2 = np.zeros_like(H1)
H3 = np.zeros_like(H1)
H4 = np.zeros_like(H1)
H5 = np.zeros_like(H1)
H6 = np.zeros_like(H1)

In [6]:
m_true = np.arange(1., P+1., 1.) # gradient of parameter vector
m1 = np.copy(m_true)
m2 = np.copy(m1)
m3 = np.copy(m1)
m4 = np.copy(m1)
m5 = np.copy(m1)
m6 = np.copy(m1)

In [7]:
ml = np.zeros(M+2) # constant vector input for phi3
m2l = np.zeros(2) # constant vector input for phi4

### Constraints

Here we describe the constraints and the result from its functions.

### (1) Smoothness constraint on the adjacent radial distances within each prism $\varphi_{1}(\textbf{m})$

Smoothness constraint on the estimated radial distances within each prism. This constraint imposes that the $j$-th radial distance of the $k$-th prism $r^{k}_{j}$ must be close to the adjacent one $r^{k}_{j+1}$ within the same $k$-th prism. It prevents descontinuities on the estimated radial distances within each prism and imposes a cylindrical shape to the prisms. So, the squared Euclidian Norm of the difference between the adjacent radial distances must be minimun. This constraint can be described by

\begin{equation}
\varphi_{1}(\textbf{m}) = \sum\limits^{L}_{k=1}\left[\left(r^{k}_{M}-r^{k}_{1}\right)^2 + \sum\limits^{M-1}_{j=1}\left(r^{k}_{j}-r^{k}_{j+1}\right)^2\right]
\end{equation}

The geometric representation of the constraint $\varphi_{1}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi1.png",width=500>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
\varphi_{1}(\textbf{m}) = \textbf{m}^{T}\textbf{R}^{T}_{1}\textbf{R}_{1}\textbf{m}
\end{equation}

where

\begin{equation}
\textbf{R}_{1} = 
\begin{bmatrix}
\textbf{R}^{\sharp} & 0 & \cdots & 0 \\
0 & \textbf{R}^{\sharp} &  \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 &  \cdots & \textbf{R}^{\sharp} \\
\end{bmatrix}_{ML\times P}
\end{equation}

while

\begin{equation}
\textbf{R}^{\sharp} = 
\begin{bmatrix}
1 & -1 & 0 & \cdots & 0 & 0 & 0 & 0 \\
0 & 1 & -1 & \cdots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 1 & -1 & 0 & 0 \\
-1 & 0 & 0 & \cdots & 0 & 1 & 0 & 0 \\
\end{bmatrix}_{M\times(M+2)}
\end{equation}

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{1}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{1}(\textbf{m}) = \textbf{R}^{T}_{1}\textbf{R}_{1}\textbf{m}
\end{equation}
\begin{equation}
\nabla^2\varphi_{1}(\textbf{m}) = \textbf{R}^{T}_{1}\textbf{R}_{1}
\end{equation}

### creating matrices

In [8]:
RH = np.zeros((M, M+2))
i = np.arange(M)
RH[i,i] = 1.
RH[i[:M-1],i[:M-1]+1] = -1.
RH[-1,0] = -1.
print RH

[[ 1. -1.  0.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.]
 [-1.  0.  0.  1.  0.  0.]]


In [9]:
R1 = np.zeros((M*L,P))
for i in range(M-1):
    R1[i*M:(i+1)*M,i*(M+2):(i+1)*(M+2)] = RH

print R1

[[ 1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  0.]]


### calculating by matrix multiplication

In [10]:
H1_matrix = np.dot(R1.T,R1)
grad1_matrix = np.dot(H1_matrix,m1)
phi1_matrix = np.dot(m1,grad1_matrix)

In [11]:
print H1_matrix

[[ 2. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0. -1.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0. -1.  2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2. -1.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [12]:
print grad1_matrix + m1

[ -3.   2.   3.   8.   5.   6.   3.   8.   9.  14.  11.  12.   9.  14.  15.
  20.  17.  18.]


In [13]:
print phi1_matrix

36.0


### calculating by my functions

In [14]:
grad1 = mfun.gradient_phi_1(M,L,m1,alpha)
H1 = mfun.Hessian_phi_1(M,L,H1,alpha)
phi1 = mfun.phi_1(M, L, m1, alpha)

In [15]:
print grad1

[ -3.   2.   3.   8.   5.   6.   3.   8.   9.  14.  11.  12.   9.  14.  15.
  20.  17.  18.]


In [16]:
print H1

[[ 2. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0. -1.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0. -1.  2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2. -1.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [17]:
print phi1

36.0


### comparing the results

In [18]:
np.allclose(H1, H1_matrix)

True

In [19]:
np.allclose(grad1, grad1_matrix + m1)

True

In [20]:
np.allclose(phi1, phi1_matrix)

True

### (2) Smoothness constraint on the adjacent radial distances of adjacent prism $\varphi_{2}(\textbf{m})$

This constraint imposes that the squared Euclidian norm of diference between adjacent radial distances ($r_j^k$ e $r_j^{k+1}$) within vertically adjacent prisms must be minimum. It prevents abrupt descontinuities on the estimated adjacent radial distances within adjacent prism that forces all estimated prism to have a similar shape. Mathematically, it can be described by

\begin{equation}
\varphi_{2}(\textbf{m}) = \sum\limits^{L-1}_{k=1}\left[\sum\limits^{M}_{j=1}\left(r^{k+1}_{j}-r^{k}_{j}\right)^2\right]
\end{equation}

The geometric representation of the constraint $\varphi_{2}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi2.png",width=400>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
    \varphi_{2}(\textbf{m}) = \textbf{m}^{T}\textbf{R}^{T}_{2}\textbf{R}_{2}\textbf{m}
\end{equation}

where

\begin{equation}
    \textbf{R}_{2} = 
    \begin{bmatrix}
        \textbf{R}^{-}_{2} & \textbf{R}^{+}_{2} & 0 & \cdots & 0 & 0 \\
        0 & \textbf{R}^{-}_{2} & \textbf{R}^{+}_{2} & \cdots & 0 & 0 \\
        \vdots & \vdots & \vdots & & \vdots & \vdots \\
        0 & 0 & 0 & \cdots & \textbf{R}^{-}_{2} & \textbf{R}^{+}_{2} \\
    \end{bmatrix}_{M(L-1)\times P}
\end{equation}

while

\begin{equation}
    \textbf{R}^{-}_{2} = 
    \begin{bmatrix}
        -1 & 0 & \cdots & 0 & 0 & 0 \\
        0 & -1 & \cdots & 0 & 0 & 0 \\
        \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
        0 & 0 & \cdots & -1 & 0  & 0 \\
    \end{bmatrix}_{M\times (M+2)}
\end{equation}

and

\begin{equation}
    \textbf{R}^{+}_{2} = 
     \begin{bmatrix}
        1 & 0 & \cdots & 0 & 0 & 0 \\
        0 & 1 & \cdots & 0 & 0 & 0 \\
        \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
        0 & 0 & \cdots & 1 & 0 & 0 \\
    \end{bmatrix}_{M\times (M+2)}
\end{equation}

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{2}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{2}(\textbf{m}) = \textbf{R}^{T}_{2}\textbf{R}_{2}\textbf{m}
\end{equation}
\begin{equation}
\nabla^2\varphi_{2}(\textbf{m}) = \textbf{R}^{T}_{2}\textbf{R}_{2}
\end{equation}

### creating matrices

In [21]:
R2M = np.zeros((M,M+2))
i = np.arange(M)
R2M[i,i] = -1.
print R2M

[[-1.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.]]


In [22]:
R2P = np.zeros((M,M+2))
R2P[i,i] = 1.
print R2P

[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]]


In [23]:
R2 = np.zeros((M*(L-1),P))
for i in range(M-2):
    R2[i*M:(i+1)*M,i*(M+2):(i+1)*(M+2)] = R2M
    R2[i*M:(i+1)*M,(i+1)*(M+2):(i+2)*(M+2)] = R2P
print R2

[[-1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.]]


### calculating by matrix multiplication

In [24]:
H2_matrix = np.dot(R2.T,R2)
grad2_matrix = np.dot(H2_matrix,m2)
phi2_matrix = np.dot(m2,grad2_matrix)

In [25]:
print H2_matrix

[[ 1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [26]:
print grad2_matrix + m2

[ -5.  -4.  -3.  -2.   5.   6.   7.   8.   9.  10.  11.  12.  19.  20.  21.
  22.  17.  18.]


In [27]:
print phi2_matrix

288.0


### calculating by my functions

In [28]:
grad2 = mfun.gradient_phi_2(M,L,m2,alpha)
H2 = mfun.Hessian_phi_2(M,L,H2,alpha) 
phi2 = mfun.phi_2(M, L, m2, alpha)

In [29]:
print grad2

[ -5.  -4.  -3.  -2.   5.   6.   7.   8.   9.  10.  11.  12.  19.  20.  21.
  22.  17.  18.]


In [30]:
print H2

[[ 1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [31]:
print phi2

288.0


### comparing the results

In [32]:
np.allclose(H2, H2_matrix)

True

In [33]:
np.allclose(grad2, grad2_matrix + m2)

True

In [34]:
np.allclose(phi2, phi2_matrix)

True

### (3) The constraint of outcropping source $\varphi_{3}(\textbf{m})$

This constraint imposes that the estimated horizontal face of the shallowest prism must be close to the known outcropping boundary. The outcropping boundary must be describe by a polygon, whose radial distances and the Cartesian coordinates of the origin are $r_j^0$, $x_0^0$ and $y_0^0$. The shallowest prism is described by $r_j^1$, $x_j^1$ and $y_j^0$. So, the constraint imposes that a minimum value to the squared Euclidian norm between $r_j^0$ and $r_j^1$ and the squared Euclidian norm between the horizontal coordinates ($x_0^1$,$y_0^1$) and ($x_0^0$,$y_0^0$):

\begin{equation}
\varphi_{3}(\textbf{m}) = \left[\left(x_{0}^{1} - x_{0}^{0}\right)^2 + \left(y_{0}^{1} - y_{0}^{0}\right)^2 + \sum\limits^{M}_{j=1}\left(r^{1}_{j}-r^{0}_{j}\right)^2\right]
\end{equation}

The geometric representation of the constraint $\varphi_{3}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi3.png",width=400>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
\varphi_{3}(\textbf{m}) = \left(\textbf{A}\textbf{m} - \textbf{m}"\right)^{T}\left(\textbf{A}\textbf{m} - \textbf{m}"\right)
\end{equation}

where

\begin{equation}
\textbf{m}" = 
\begin{bmatrix}
r_{1}^{0} \\
r_{2}^{0} \\
\vdots \\
r_{M}^{0} \\
x_{0}^{0} \\
y_{0}^{0}
\end{bmatrix}_{P\times 1}
\end{equation}

and

\begin{equation}
\textbf{A} = 
\begin{bmatrix}
\textbf{I} & 0 \\
\end{bmatrix}_{(M+2)\times P}
\end{equation}

while

$\textbf{I}$ is the identity of order $M+2$.

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{3}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{3}(\textbf{m}) = \textbf{A}^{T}\left(\textbf{A}\textbf{m} - \textbf{m}"\right)
\end{equation}
\begin{equation}
\nabla^2\varphi_{3}(\textbf{m}) = \textbf{A}^{T}\textbf{A}
\end{equation}

### creating matrices

In [35]:
A = np.zeros((M+2,P))
I = np.identity(M+2)
A[:,:M+2] = I
print A

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


### calculating by matrix multiplication

In [36]:
H3_matrix = np.dot(A.T,A)
grad3_matrix = np.dot(A.T,np.dot(A,m3)-ml)
phi3_matrix = np.dot(np.dot(A,m3)-ml,np.dot(A,m3)-ml)

In [37]:
print H3_matrix

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [38]:
print grad3_matrix + m_true

[  2.   4.   6.   8.  10.  12.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.]


In [39]:
print phi3_matrix

91.0


### calculating by my functions

In [40]:
H3 = mfun.Hessian_phi_3(M,L,H3,alpha) 
phi3 = mfun.phi_3(M, L, m3, ml, alpha)
grad3 = mfun.gradient_phi_3(M,L,m3,ml,alpha)

In [41]:
print grad3

[  2.   4.   6.   8.  10.  12.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.]


In [42]:
print H3

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [43]:
print phi3

91.0


### comparing the results

In [44]:
np.allclose(H3, H3_matrix)

True

In [45]:
np.allclose(grad3, grad3_matrix + m_true)

True

In [46]:
np.allclose(phi3, phi3_matrix)

True

### (4) The constraint of horizontal location of outcropping source $\varphi_{4}(\textbf{m})$

This constraint is an special case from $\varphi_{3}(\textbf{m})$ that only the Cartesian coordinates of the origin of th outcropping body are known. So, this constraint imposes that the squared Euclidian norm between the ($x_0^1$,$y_0^1$) and ($x_0^0$,$y_0^0$):

\begin{equation}
\varphi_{4}(\textbf{m}) = \left[\left(x_{0}^{1} - x_{0}^{0}\right)^2 + \left(y_{0}^{1} - y_0^0\right)^2\right]
\end{equation}

The geometric representation of the constraint $\varphi_{4}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi4.png",width=400>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
\varphi_{4}(\textbf{m}) = \left(\textbf{B}\textbf{m} - \textbf{m}'\right)^{T}\left(\textbf{B}\textbf{m} - \textbf{m}'\right)
\end{equation}

where
\begin{equation}
\textbf{B} = 
\begin{bmatrix}
\textbf{B}^{\sharp} & 0 \\
\end{bmatrix}_{2\times P}
\end{equation}

while

\begin{equation}
\textbf{B}^{\sharp} = 
\begin{bmatrix}
0 & \cdots & 0 & 1 & 0 \\
0 & \cdots & 0 & 0 & 1
\end{bmatrix}_{2\times (M+2)}
\end{equation}

and

\begin{equation}
\textbf{m}' = 
\begin{bmatrix}
x^0_0 \\
y^0_0
\end{bmatrix}_{2\times 1}
\end{equation}

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{4}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{4}(\textbf{m}) = \textbf{B}^{T}\left(\textbf{B}\textbf{m} - \textbf{m}'\right)
\end{equation}
\begin{equation}
\nabla^2\varphi_{4}(\textbf{m}) = \textbf{B}^{T}\textbf{B}
\end{equation}

### creating matrices

In [47]:
B = np.zeros((2,P))
BH = np.zeros((2,M+2))
BH[0,-2] = 1.
BH[1,-1] = 1.
B[:,:M+2] = BH
print B

[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


### calculating by matrix multiplication

In [48]:
H4_matrix = np.dot(B.T,B)
grad4_matrix = np.dot(B.T,np.dot(B,m4)-m2l)
phi4_matrix = np.dot(np.dot(B,m4)-m2l,np.dot(B,m4)-m2l)

In [49]:
print H4_matrix

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [50]:
print grad4_matrix + m_true

[  1.   2.   3.   4.  10.  12.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.]


In [51]:
print phi4_matrix

61.0


### calculating by my functions

In [52]:
H4 = mfun.Hessian_phi_4(M,L,H4,alpha) 
phi4 = mfun.phi_4(M, L, m4, m2l, alpha)
grad4 = mfun.gradient_phi_4(M,L,m4,m2l,alpha) 

In [53]:
print grad4

[  1.   2.   3.   4.  10.  12.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.]


In [54]:
print H4

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [55]:
print phi4

61.0


### comparing the results

In [56]:
np.allclose(H4, H4_matrix)

True

In [57]:
np.allclose(grad4, grad4_matrix + m_true)

True

In [58]:
np.allclose(phi4, phi4_matrix)

True

### (5) Smoothness constraint on the origins of adjacent prisms $\varphi_{5}(\textbf{m})$

This constraint imposses a minimum value to the difference between horizontal Cartesian coordinates of vertically adjacent prisms. It prevents descontinuities between estimated horizontal Cartesian coordinates of vertically adjacent prisms. This constraint forces smooth horizontal displacements between all vertically adjacent prisms. It cand be described by

\begin{equation}
\varphi_{5}(\textbf{m}) = \sum\limits^{L-1}_{k=1}\left[\left(x_{0}^{k+1} - x_{0}^{k}\right)^2 + \left(y_{0}^{k+1} - y_{0}^{k}\right)^2 \right]
\end{equation}

The geometric representation of the constraint $\varphi_{5}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi5.png",width=400>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
\varphi_{5}(\textbf{m}) = \textbf{m}^{T}\textbf{R}^{T}_{5}\textbf{R}_{5}\textbf{m}
\end{equation}

where
\begin{equation}
\textbf{R}_{5} = 
\begin{bmatrix}
\textbf{R}^{-}_{5} & \textbf{R}^{+}_{5} & 0 & \cdots & 0 & 0 \\
0 & \textbf{R}^{-}_{5} & \textbf{R}^{+}_{5} & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & & \vdots & \vdots \\
0 & 0 & 0 & \cdots & \textbf{R}^{-}_{5} & \textbf{R}^{+}_{5} \\
\end{bmatrix}_{2(L-1)\times P}
\end{equation}

while

\begin{equation}
\textbf{R}^{-}_{5} = 
\begin{bmatrix}
0 & 0 & \cdots & 0 & -1 & 0 \\
0 & 0 & \cdots & 0 & 0 & -1 \\
\end{bmatrix}_{2\times (M+2)}
\end{equation}

and

\begin{equation}
\textbf{R}^{+}_{5} = 
\begin{bmatrix}
0 & 0 & \cdots & 0 & 1 & 0 \\
0 & 0 & \cdots & 0 & 0 & 1 \\
\end{bmatrix}_{2\times (M+2)}
\end{equation}

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{5}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{5}(\textbf{m}) = \textbf{R}^{T}_{5}\textbf{R}_{5}\textbf{m}
\end{equation}
\begin{equation}
\nabla^2\varphi_{5}(\textbf{m}) = \textbf{R}^{T}_{5}\textbf{R}_{5}
\end{equation}

### creating matrices

In [59]:
R5M = np.zeros((2,M+2))
R5M[0,-2] = -1.
R5M[1,-1] = -1.
print R5M

[[ 0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0. -1.]]


In [60]:
R5P = np.zeros((2,M+2))
R5P[0,-2] = 1.
R5P[1,-1] = 1.
print R5P

[[ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.]]


In [61]:
R5 = np.zeros((2*(L-1),P))
for i in range(M-2):
    R5[i*2:(i+1)*2,i*(M+2):(i+1)*(M+2)] = R5M
    R5[i*2:(i+1)*2,(i+1)*(M+2):(i+2)*(M+2)] = R5P
print R5

[[ 0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.]]


### calculating by matrix multiplication

In [62]:
H5_matrix = np.dot(R5.T,R5)
grad5_matrix = np.dot(np.dot(R5.T,R5),m5)
phi5_matrix = np.dot(grad5_matrix,m5)

In [63]:
print H5_matrix

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [64]:
print grad5_matrix + m5

[  1.   2.   3.   4.  -1.   0.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  23.  24.]


In [65]:
print phi5_matrix

144.0


### calculating by my functions

In [66]:
grad5 = mfun.gradient_phi_5(M,L,m5,alpha)
H5 = mfun.Hessian_phi_5(M,L,H5,alpha) 
phi5 = mfun.phi_5(M, L, m5, alpha)

In [67]:
print grad5

[  1.   2.   3.   4.  -1.   0.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  23.  24.]


In [68]:
print H5

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [69]:
print phi5

144.0


### comparing the results

In [70]:
np.allclose(H5, H5_matrix)

True

In [71]:
np.allclose(grad5, grad5_matrix + m5)

True

In [72]:
np.allclose(phi5, phi5_matrix)

True

### (6) Minimum Euclidian norm constraint on the adjacent radial distances within each prism $\varphi_{6}(\textbf{m})$

This constraint imposes that the squared Euclidian norm of the radial distances must be minimum. It implies that the estimated radial distances to be close to zero. Mathematically, it can be described as:

\begin{equation}
\varphi_{6}(\textbf{m}) = \sum\limits^{L}_{k=1}\sum\limits^{M}_{j=1}\left(r_{j}^{k}\right)^2
\end{equation}

The geometric representation of the constraint $\varphi_{6}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi6.png",width=400>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
\varphi_{6}(\textbf{m}) = \textbf{m}^{T}\textbf{C}^{T}\textbf{C}\textbf{m}
\end{equation}

where

\begin{equation}
\textbf{C} = 
\begin{bmatrix}
\textbf{C}^{\sharp} & 0 & \cdots & 0 \\
0 & \textbf{C}^{\sharp} &  \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 &  \cdots & \textbf{C}^{\sharp} \\
\end{bmatrix}_{P\times P}
\end{equation}

while



\begin{equation}
\textbf{C}^{\sharp} = 
\begin{bmatrix}
1 & & & & \\
 & \ddots &  & &  \\
 &  & 1 & & \\
 &  & & 0 &  \\
 &  & & & 0 \\
\end{bmatrix}_{(M+2)\times (M+2)}
\end{equation}

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{6}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{5}(\textbf{m}) = \textbf{C}^{T}\textbf{C}\textbf{m}
\end{equation}
\begin{equation}
\nabla^2\varphi_{5}(\textbf{m}) = \textbf{C}^{T}\textbf{C}
\end{equation}

### creating matrices

In [73]:
CH = np.zeros((M+2,M+2))
i = np.arange(M)
CH[i,i] = 1.
print CH

[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]


In [74]:
C = np.zeros((P,P))
for i in range(0,P,M+2):
    C[i:i+M+2,i:i+M+2] = CH
print C

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

### calculating by matrix multiplication

In [75]:
H6_matrix = np.dot(C.T,C)
grad6_matrix = np.dot(np.dot(C.T,C),m6)
phi6_matrix = np.dot(grad6_matrix, m6)

In [76]:
print H6_matrix

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [77]:
print grad6_matrix +m6

[  2.   4.   6.   8.   5.   6.  14.  16.  18.  20.  11.  12.  26.  28.  30.
  32.  17.  18.]


In [78]:
print phi6_matrix

1170.0


### calculating by my functions

In [79]:
H6 = mfun.Hessian_phi_6(M,L,H6,alpha) 
phi6 = mfun.phi_6(M, L, m6, alpha)
grad6 = mfun.gradient_phi_6(M,L,m6,alpha)

In [80]:
print grad6

[  2.   4.   6.   8.   5.   6.  14.  16.  18.  20.  11.  12.  26.  28.  30.
  32.  17.  18.]


In [81]:
print H6

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [82]:
print phi6

1170.0


### comparing the results

In [83]:
np.allclose(H6, H6_matrix)

True

In [84]:
np.allclose(grad6, grad6_matrix + m6)

True

In [85]:
np.allclose(phi6, phi6_matrix)

True